In [2]:
#create MS-Excel table with names of cat pics, so Matt could write there text from these

import xlwt

names = os.listdir("./memes/cat/")

wb = xlwt.Workbook()
ws = wb.add_sheet('pictures')

i = 1
for name in names:
    ws.write(i,0,name)
    i += 1

wb.save('test.xls')


In [1]:
from scipy.misc import imrotate
from skimage.transform import resize
from imageio import imread
import matplotlib.pyplot as plt
%matplotlib inline
import os
import numpy as np
from sklearn.utils import shuffle
import tqdm


In [2]:
#read images from folder

def make_images(data_dir):
    img_names = os.listdir(data_dir)
    img_paths = list(map(lambda fname: os.path.join(data_dir,fname),img_names))
    img = imread(img_paths[100])
    
    standard_size = (200,200)

    resized_images = []
    for i in range(len(img_names)) :
        resized_images.append(resize(imread(img_paths[i]), standard_size))

    resized_images = np.array(resized_images)
    
    sh = resized_images.shape
    print (sh[0], " images ", sh[1], "x", sh[2], "rgb(", sh[3], ")")
    return resized_images

In [3]:
X_cats = make_images("./memes/cat/")
X_tind = make_images("./memes/tinder/")

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


829  images  200 x 200 rgb( 3 )
435  images  200 x 200 rgb( 3 )


In [4]:
Y_cats = np.zeros(X_cats.shape[0])
Y_tind = np.ones(X_tind.shape[0])
print(Y_cats.shape, Y_tind.shape)

(829,) (435,)


In [5]:
X = np.concatenate((X_cats, X_tind))
Y = np.concatenate((Y_cats, Y_tind))
print(X.shape, Y.shape)

(1264, 200, 200, 3) (1264,)


In [6]:
X1 = []
i = 0
for image in tqdm.tqdm(X):
    X1.append([])
    for row in image:
        for col in row:
            X1[i].append(col[0])
            X1[i].append(col[1])
            X1[i].append(col[2])
    i += 1
             
X = np.array(X1)

100%|██████████████████████████████████████████████████████████████████████████████| 1264/1264 [02:33<00:00,  8.24it/s]


In [7]:
X.shape, Y.shape

((1264, 120000), (1264,))

In [9]:
from sklearn.model_selection import train_test_split

np.random.seed(len(X))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [10]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((1011, 120000), (1011,), (253, 120000), (253,))

In [12]:
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier

bag = BaggingClassifier().fit(X_train,Y_train)
#ext = ExtraTreesClassifier().fit(X_train,Y_train)
#grb = GradientBoostingClassifier().fit(X_train,Y_train)

y_bag = bag.predict(X_test)
#y_ext = ext.predict(X_test)
#y_grb = grb.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score, roc_auc_score

print("accuracy", accuracy_score(Y_test, y_bag))
print("roc auc ", roc_auc_score(Y_test, y_bag))

#print("ex tree: accuracy", accuracy_score(Y, y_ext))
#print("grboost: accuracy", accuracy_score(Y, y_grb))

accuracy 0.9525691699604744
roc auc  0.9457736877982277


In [16]:
ext = ExtraTreesClassifier().fit(X_train,Y_train)
#grb = GradientBoostingClassifier().fit(X_train,Y_train)

y_ext = ext.predict(X_test)
#y_grb = grb.predict(X_test)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [18]:

print("accuracy", accuracy_score(Y_test, y_ext))
print("roc auc ", roc_auc_score(Y_test, y_ext))

accuracy 0.9723320158102767
roc auc  0.96359918200409


In [19]:
grb = GradientBoostingClassifier().fit(X_train,Y_train)

y_grb = grb.predict(X_test)

In [20]:
print("accuracy", accuracy_score(Y_test, y_grb))
print("roc auc ", roc_auc_score(Y_test, y_grb))

accuracy 0.9802371541501976
roc auc  0.9796864349011588
